In [48]:
import pandas as pd
import numpy as np
#from cleanco import cleanco
import re
import datetime as dt
import os.path
import json
from tqdm import tqdm

In [49]:
checked_domains_file = 'CheckedCompanies_20191212.csv'

In [50]:
after_underscore = checked_domains_file.find('_')+1
before_ext = checked_domains_file.find('.csv')
checked_file_date = checked_domains_file[after_underscore:before_ext]

In [51]:
checked_file_date

'20191212'

In [52]:
## 'CheckedCompanies.csv' from industry segmentation Google Sheets document

In [53]:
checked_domains = pd.read_csv(checked_domains_file)
cols = ['Date','Company Name(s)','Email Domain','Industry Segment','Respective Sub-Industry Segment']
checked_domains = checked_domains[cols]

In [54]:
# Splitting company name(s) attribute and email domains
for i in tqdm(range(checked_domains.shape[0])):
    co_names = str(checked_domains.iloc[i][1]).lower().split(';')
    e_domains = str(checked_domains.iloc[i][2]).lower().split(',')
    checked_domains.iloc[i][1] = co_names
    checked_domains.iloc[i][2] = e_domains

100%|██████████| 1661/1661 [00:01<00:00, 1078.58it/s]


In [55]:
checked_domains['Date'] = pd.to_datetime(checked_domains['Date'],errors='coerce').dt.date

In [56]:
checked_domains.sort_values('Date',ascending=False,inplace=True)
checked_domains.reset_index(drop=True,inplace=True)

In [57]:
checked_domains.dropna(subset=['Date'],inplace=True)

In [58]:
checked_array = checked_domains.to_numpy()

In [59]:
## Removes whitespace and punctuation in company name and email lists for each company entry

In [60]:
for i in tqdm(range(checked_array.shape[0])):
    checked_array[i][0] = checked_array[i][0].strftime('%Y%m%d')
    
    domain_list = checked_array[i][2]
    for d in range(len(domain_list)):
        domain_list[d] = domain_list[d].strip()
    checked_array[i][2] = domain_list
    
    company_list = checked_array[i][1]
    for c in range(len(company_list)):
        #company_list[c] = company_list[c].translate(None, string.punctuation)
        company_list[c] = re.sub(r"[(,.;@#?!&$-)]+\ *"," ", company_list[c])
        company_list[c] = ' '.join(company_list[c].split())
        #company_list[c] = cleanco(company_list[c].strip()).clean_name()
    checked_array[i][1] = company_list

100%|██████████| 1648/1648 [00:00<00:00, 16021.48it/s]


In [61]:
checked_array[1500]

array(['20190703', list(['schneider logistics']),
       list(['@schneider.com']), 'Intermediary', '3PL / Brokerage'],
      dtype=object)

In [62]:
## Checks and creates dictionaries of {'domain':'[Company Info]'} and {'domain':frequency}

In [63]:
d_segment_file = 'DictDomainSegments.json'
d_duplicate_file = 'DictDomainDuplicates.json'

if os.path.exists(d_segment_file) and os.path.exists(d_duplicate_file):
    print('Dictionary files already exist.')
    with open(d_segment_file,'r+') as dsf, open(d_duplicate_file,'r+') as ddf:
        ds = json.load(dsf)
        dd = json.load(ddf)
        for i in range(checked_array.shape[0]):
            for j in range(len(checked_array[i][2])):
                domain = checked_array[i][2][j]
                if domain not in ds and domain not in dd:
                    ds[domain] = [checked_array[i][1][0]
                                 ,checked_array[i][3]
                                 ,checked_array[i][4]
                                 ,checked_array[i][0]
                                 ,checked_file_date]
                elif domain in ds and domain not in dd:
                    if ds.get(domain)[4] < checked_array[i][0] <= checked_file_date: 
                        temp = {domain:2}
                        dd.update(temp)
                
        dsf.seek(0)
        json.dump(ds,dsf)
        dsf.truncate()
        ddf.seek(0)
        json.dump(dd,ddf)
        ddf.truncate()
else:
    print('Dictionary files do not exist, they have been created.')
    with open(d_segment_file,'w') as dsf, open(d_duplicate_file,'w') as ddf:
        ds = dict()
        dd = dict()
        for i in range(checked_array.shape[0]):
            for j in range(len(checked_array[i][2])):
                domain = checked_array[i][2][j]
                if domain not in ds:
                    ds[domain] = [checked_array[i][1][0]
                                ,checked_array[i][3]
                                ,checked_array[i][4]
                                ,checked_array[i][0]
                                ,checked_file_date]
                else:
                    if domain not in dd:
                        #dd[checked_array[i][2][j]] = 2
                        temp = {domain:2}
                        dd.update(temp)
                        
                    else:
                        #dd[checked_array[i][2][j]] = dd.get(checked_array[i][2][j])+1
                        temp = {domain:dd.get(domain)+1}
                        dd.update(temp)
        json.dump(ds,dsf)
        json.dump(dd,ddf)

Dictionary files do not exist, they have been created.


In [64]:
cn_segment_file = 'DictCoNameSegments.json'
cn_duplicate_file = 'DictCoNameDuplicates.json'

if os.path.exists(cn_segment_file) and os.path.exists(cn_duplicate_file):
    print('Dictionary files already exist.')
    with open(cn_segment_file,'r+') as cnf, open(cn_duplicate_file,'r+') as cndf:
        cnd = json.load(cnf)
        cndd = json.load(cndf)
        for i in range(checked_array.shape[0]):
            for j in range(len(checked_array[i][1])):
                coname = checked_array[i][1][j]
                coname = re.sub(r"[(,.;@#?!&$-)]+\ *"," ",coname)
                coname = ' '.join(coname.split())
                if coname not in cnd and coname not in cndd:
                    cnd[coname] = [checked_array[i][3]
                                 ,checked_array[i][4]
                                 ,checked_file_date]
                elif coname in cnd and coname not in cndd:
                    if cnd.get(coname)[2] < checked_array[i][0] <= checked_file_date: 
                        temp = {coname:2}
                        cndd.update(temp)
        cnf.seek(0)
        json.dump(cnd,cnf)
        cnf.truncate()
        cndf.seek(0)
        json.dump(cndd,cndf)
        cndf.truncate()
else:
    print('Dictionary files do not exist, they have been created.')
    with open(cn_segment_file,'w') as cnf, open(cn_duplicate_file,'w') as cndf:
        cnd = dict()
        cndd = dict()
        for i in range(checked_array.shape[0]):
            for j in range(len(checked_array[i][1])):
                coname = checked_array[i][1][j]
                coname = re.sub(r"[(,.;@#?!&$-)]+\ *"," ",coname)
                coname = ' '.join(coname.split())
                if coname not in cnd:
                    cnd[coname] = [checked_array[i][3]
                                 ,checked_array[i][4]
                                 ,checked_file_date]
                else:
                    if coname not in cndd:
                        #dd[checked_array[i][2][j]] = 2
                        temp = {coname:2}
                        cndd.update(temp)
                        
                    else:
                        #dd[checked_array[i][2][j]] = dd.get(checked_array[i][2][j])+1
                        temp = {coname:cndd.get(coname)+1}
                        cndd.update(temp)
        json.dump(cnd,cnf)
        json.dump(cndd,cndf)

Dictionary files do not exist, they have been created.


In [65]:
print('DictDomainSegments: {0}\nDictDomainDuplicates: {1}\nDictCoNameSegments: {2}\nDictCoNameDuplicates: {3}'.format(len(ds.keys()),len(dd.keys()),len(cnd.keys()),len(cndd.keys())))

DictDomainSegments: 2205
DictDomainDuplicates: 46
DictCoNameSegments: 2229
DictCoNameDuplicates: 0


In [66]:
## Removes keys in the duplicate dictionary from the segmentation dictionaries

In [67]:
dd_keys = list(dd.keys())
for i in range(len(dd_keys)):
    if dd_keys[i] in ds:
        del ds[dd_keys[i]]

In [68]:
cndd_keys = list(cndd.keys())
for i in range(len(cndd_keys)):
    if cndd_keys[i] in cnd:
        del cnd[cndd_keys[i]]

In [69]:
## Udpates json files with dictionary changes

In [70]:
with open(d_segment_file,'w') as dsf, open(d_duplicate_file,'w') as ddf, open(cn_segment_file,'w') as cnf, open(cn_duplicate_file,'w') as cndf:
    #f.seek(0)
    json.dump(ds,dsf)
    #f.truncate()
    #ddf.seek(0)
    json.dump(dd,ddf)
    #ddf.truncate()
    json.dump(cnd,cnf)
    json.dump(cndd,cndf)

In [71]:
print('DictDomainSegments: {0}\nDictDomainDuplicates: {1}\nDictCoNameSegments: {2}\nDictCoNameDuplicates: {3}'.format(len(ds.keys()),len(dd.keys()),len(cnd.keys()),len(cndd.keys())))

DictDomainSegments: 2159
DictDomainDuplicates: 46
DictCoNameSegments: 2229
DictCoNameDuplicates: 0
